1. Correlation coefficient
<br>pd.df.corr(method='pearson', numeric_only=False)
<br>options for the method : pearson (default), kendall, spearman
<br>options of the numeric_only : False (default), True

In [1]:
# as an example, consider students' height and weight
# turn the data to a dataframe to apply .corr()
import pandas as pd
data = pd.DataFrame({'height':[150, 160, 170, 175, 165], 'weight':[42, 50, 70, 64, 56]})
data.corr()

,height,weight
height,1.000000,0.919509
weight,0.919509,1.000000


In [3]:
# to select a certain value, use .iloc[row, col]
data.corr().iloc[0,1]

np.float64(0.9195090879163764)

In [4]:
# try another method
print(data.corr(method='kendall'))
print(data.corr(method='spearman'))

        height  weight
height     1.0     0.8
weight     0.8     1.0
        height  weight
height     1.0     0.9
weight     0.9     1.0


In [8]:
# t-test on correlation coefficients
from scipy import stats
print(stats.pearsonr(data['height'], data['weight']))
print(stats.spearmanr(data['height'], data['weight']))
print(stats.kendalltau(data['height'], data['weight']))

PearsonRResult(statistic=np.float64(0.9195090879163766), pvalue=np.float64(0.02707945689558949))
SignificanceResult(statistic=np.float64(0.8999999999999998), pvalue=np.float64(0.03738607346849874))
SignificanceResult(statistic=np.float64(0.7999999999999999), pvalue=np.float64(0.08333333333333333))


2. Simple linear regression

In [44]:
data = {
    'weight': [42, 50, 70, 64, 56, 48, 68, 60, 65, 52,
        54, 67, 49, 51, 58, 55, 69, 61, 66, 53], # input
    'height': [150, 160, 170, 175, 165, 155, 172, 168, 174, 158, 
          162, 173, 156, 159, 167, 163, 171, 169, 176, 161] # output
}

from statsmodels.formula.api import ols
model = ols('height ~ weight', data).fit() # y ~ x
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 height   R-squared:                       0.892
Model:                            OLS   Adj. R-squared:                  0.886
Method:                 Least Squares   F-statistic:                     148.0
Date:                Thu, 28 Nov 2024   Prob (F-statistic):           4.04e-10
Time:                        13:19:11   Log-Likelihood:                -45.761
No. Observations:                  20   AIC:                             95.52
Df Residuals:                      18   BIC:                             97.51
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    115.0676      4.158     27.671      0.000     106.331     123.804
weight         0.8658      0.071     12.167      0.000       0.716       1.015
==============================================================================
Omnibus:                        0.985   Durbin-Watson:                   2.609
Prob(Omnibus):                  0.611   Jarque-Bera (JB):                0.336
Skew:                          -0.315   Prob(JB):                        0.845
Kurtosis:                       3.082   Cond. No.                         432.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [45]:
# to print the R-squared value only:
model.rsquared

np.float64(0.8915914350087261)

In [46]:
# to access the regression model
model.params

Intercept    115.067639
weight         0.865844
dtype: float64

In [47]:
print(model.params['weight'])
print(model.params['Intercept'])

0.8658438852380184
115.06763904471866


In [48]:
model.pvalues

Intercept    3.335096e-16
weight       4.037933e-10
dtype: float64

In [49]:
# to change the unit, there are two options :
print(format(model.pvalues['weight'], ".10f"))
print("{:.10f}".format(model.pvalues['weight']))

0.0000000004
0.0000000004


In [50]:
# predict a value
new_data = pd.DataFrame({'weight':[50]})
model.predict(new_data)[0]

np.float64(158.35983330661958)

In [52]:
model.predict(df['weight'])

0     151.433082
1     158.359833
2     175.676711
3     170.481648
4     163.554897
5     156.628146
6     173.945023
7     167.018272
8     171.347492
9     160.091521
10    161.823209
11    173.079179
12    157.493989
13    159.225677
14    165.286584
15    162.689053
16    174.810867
17    167.884116
18    172.213335
19    160.957365
dtype: float64

In [53]:
df = pd.DataFrame(data)
df['predict'] = model.predict(df) # same with model.predict(df['weight'])
df

,weight,height,predict
0,42,150,151.433082
1,50,160,158.359833
2,70,170,175.676711
3,64,175,170.481648
4,56,165,163.554897
5,48,155,156.628146
6,68,172,173.945023
7,60,168,167.018272
8,65,174,171.347492
9,52,158,160.091521


In [54]:
# error is defined as the original values subtracted by prediction values
df['error'] = df['height'] - df['predict']
df.head()

,weight,height,predict,error
0,42,150,151.433082,-1.433082
1,50,160,158.359833,1.640167
2,70,170,175.676711,-5.676711
3,64,175,170.481648,4.518352
4,56,165,163.554897,1.445103


In [56]:
# SSE (Sum of Squareds of Error)
sum(df['error']**2)

113.74226638884454

In [57]:
# MSE (Mean Squared Error)
MSE = (df['error']**2).mean()
MSE

np.float64(5.687113319442227)

In [59]:
# alternatively, use sklearn.metrics
from sklearn.metrics import mean_squared_error
pred = model.predict(df['weight'])
mse = mean_squared_error(df['height'], pred)
mse

np.float64(5.687113319442227)

3. Multiple linear regression
<br>y = a + b1*x + b2 * x + b3 * x + error

In [60]:
# build a multiple linear regression model of sales w.r.t the others
data = {
    'sales': [300, 320, 250, 360, 315, 328, 310, 335, 326, 280,
            290, 300, 315, 328, 310, 335, 300, 400, 500, 600], # output
    'advertisement_fee': [70, 75, 30, 80, 72, 77, 70, 82, 70, 80,
            68, 90, 72, 77, 70, 82, 40, 20, 75, 80], # input 1
    'num_employees': [15, 16, 14, 20, 19, 17, 16, 19, 15, 20,
            14, 5, 16, 17, 16, 14, 30, 40, 10, 50] # input 2
    }

In [61]:
model = ols('sales ~ advertisement_fee + num_employees', data).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sales   R-squared:                       0.512
Model:                            OLS   Adj. R-squared:                  0.454
Method:                 Least Squares   F-statistic:                     8.907
Date:                Thu, 28 Nov 2024   Prob (F-statistic):            0.00226
Time:                        13:32:33   Log-Likelihood:                -108.22
No. Observations:                  20   AIC:                             222.4
Df Residuals:                      17   BIC:                             225.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           101.0239     71.716      1.409      0.177     -50.284     252.331
advertisement_fee     1.8194      0.807      2.255      0.038       0.117       3.522
num_employees         5.9288      1.430      4.147      0.001       2.912       8.945
==============================================================================
Omnibus:                       30.534   Durbin-Watson:                   1.354
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               64.655
Skew:                           2.444   Prob(JB):                     9.13e-15
Kurtosis:                      10.327   Cond. No.                         401.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [64]:
# correlation coefficient between adv_fee and sales
# to get a correlation coefficient between two variables only,
df = pd.DataFrame(data)
df['sales'].corr(df['advertisement_fee'])

np.float64(0.13316981737040345)

In [65]:
# t-test of the correlation coefficient
stats.pearsonr(df['sales'], df['advertisement_fee']) # have correlation !

PearsonRResult(statistic=np.float64(0.1331698173704034), pvalue=np.float64(0.5756778801904271))

In [66]:
# info about the regression model
model.params

Intercept            101.023872
advertisement_fee      1.819427
num_employees          5.928756
dtype: float64

That is, sales = 101.023872 + 1.819427 * advertisement_fee + 5.928756 * num_employees

In [68]:
# get a pvalue of the coefficients,
model.pvalues

Intercept            0.176962
advertisement_fee    0.037644
num_employees        0.000675
dtype: float64

In [70]:
# predict sales given data; advertisement_fee = 50, num_employees = 20
new_data = pd.DataFrame({'advertisement_fee':[50], 'num_employees':[20]})
model.predict(new_data)[0]

np.float64(310.57033031815365)

In [73]:
df.head()

,sales,advertisement_fee,num_employees
0,300,70,15
1,320,75,16
2,250,30,14
3,360,80,20
4,315,72,19


In [76]:
df['pred'] = model.predict(df)
df['error'] = df['sales'] - df['pred']
# SSE
(df['error']**2).sum()

np.float64(58686.17827156107)

In [77]:
from sklearn.metrics import mean_squared_error
pred = model.predict(df)
mse = mean_squared_error(df['sales'], pred)
mse

np.float64(2934.3089135780533)

In [78]:
# 95% confidence interval
model.conf_int(alpha=0.05)

,0,1
Intercept,-50.283684,252.331429
advertisement_fee,0.116785,3.522069
num_employees,2.912406,8.945105


In [80]:
# given advertisement_fee = 42, num_employees = 22, find 95% confidence and prediction intervals
new_data = pd.DataFrame({'advertisement_fee':[45], 'num_employees':[22]})
pred = model.get_prediction(new_data)
pred.summary_frame(alpha=0.05)

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,313.330707,22.502058,265.855514,360.8059,180.58875,446.072663


4. Categorical variables
<br>statsmodels.api.ols() recognizes categorical data and automatically apply one-hot encoding of them.

In [82]:
study = pd.read_csv('data/study.csv')
study.head()

,study_hours,material_type,score
0,71,강의,95
1,34,독학,63
2,91,도서,95
3,80,독학,80
4,40,강의,79


In [83]:
# regression of score based on the others
from statsmodels.formula.api import ols
model = ols('score ~ study_hours + material_type', study).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  score   R-squared:                       0.969
Model:                            OLS   Adj. R-squared:                  0.968
Method:                 Least Squares   F-statistic:                     991.9
Date:                Thu, 28 Nov 2024   Prob (F-statistic):           4.42e-72
Time:                        14:13:11   Log-Likelihood:                -238.89
No. Observations:                 100   AIC:                             485.8
Df Residuals:                      96   BIC:                             496.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              59.2111      0.799     74.147      0.000      57.626      60.796
material_type[T.도서]    -8.6696      0.678    -12.778      0.000     -10.016      -7.323
material_type[T.독학]   -17.6129      0.634    -27.790      0.000     -18.871     -16.355
study_hours             0.4839      0.011     43.810      0.000       0.462       0.506
==============================================================================
Omnibus:                        1.754   Durbin-Watson:                   2.173
Prob(Omnibus):                  0.416   Jarque-Bera (JB):                1.216
Skew:                           0.231   Prob(JB):                        0.544
Kurtosis:                       3.280   Cond. No.                         228.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The regression model reads score = Intercept -8.6696 * material_type[T.도서] -17.6129 * material_type[T.독학] + 0.4839 * study_hours	